In [1]:
from typing import List
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component
from tfx.types import artifact, standard_artifacts

from module_file import _FEATURE_KEYS, _NUM_PARAM, _INPUT_DATA, _TARGET_DATA, SelectorFunc, ScoreFunc

In [18]:
class FeatureSelectionArtifact(artifact.Artifact):
  """Output artifact containing feature scores from the Feature Selection component"""
  TYPE_NAME = 'Feature Scores and P-Values'
  PROPERTIES = {
      'scores': artifact.Property(type=artifact.PropertyType.JSON_VALUE),
      'p_values': artifact.Property(type=artifact.PropertyType.JSON_VALUE)
  }

In [23]:
@component
def FeatureSelection(feature_selection: tfx.dsl.components.OutputArtifact[FeatureSelectionArtifact],
                    ):
  """Feature Selection component"""

  # Select features based on scores
  selector = SelectorFunc(ScoreFunc, k=_NUM_PARAM)
  selected_data = selector.fit_transform(_INPUT_DATA, _TARGET_DATA)

  # get scores and p-values for artifacts
  selector_scores = selector.scores_
  selector_p_values = selector.pvalues_

  # merge scores and pvalues with feature keys to create a dictionary
  selector_scores_dict = dict(zip(_FEATURE_KEYS, selector_scores))
  selector_pvalues_dict = dict(zip(_FEATURE_KEYS, selector_p_values))
    
  feature_selection.scores = selector_scores_dict
  feature_selection


  feature_selection.set_json_value_custom_property("scores", selector_scores_dict)
  feature_selection.set_json_value_custom_property("p_values", selector_pvalues_dict)

In [24]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [25]:
feature_selector = FeatureSelection()

In [26]:
context.run(feature_selector)

ExecutionResult(
    component_id: FeatureSelection
    execution_id: 1
    outputs:
        feature_selection: Channel(
            type_name: Feature Scores and P-Values
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/55/gk05zk9j6596lk4gv9p8sk680000gn/T/tfx-interactive-2021-07-29T00_33_16.018558-5g3ac5wi/FeatureSelection/feature_selection/1"
        custom_properties {
          key: "name"
          value {
            string_value: "feature_selection"
          }
        }
        custom_properties {
          key: "p_values"
          value {
            struct_value {
              fields {
                key: "petal length (cm)"
                value {
                  number_value: 5.533972277194025e-26
                }
              }
              fields {
                key: "petal width (cm)"
                value {
                  number_value: 2.7582496530034537e-15
                }
              }
              fields {
                key: "sepal length (cm)"
                value {
                  number_value: 0.004476514990225723
                }
              }
              fields {
                key: "sepal width (cm)"
                value {
                  number_value: 0.1563959804316254
                }
              }
            }
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "FeatureSelection"
          }
        }
        custom_properties {
          key: "scores"
          value {
            struct_value {
              fields {
                key: "petal length (cm)"
                value {
                  number_value: 116.3126130920702
                }
              }
              fields {
                key: "petal width (cm)"
                value {
                  number_value: 67.04836020011113
                }
              }
              fields {
                key: "sepal length (cm)"
                value {
                  number_value: 10.817820878494022
                }
              }
              fields {
                key: "sepal width (cm)"
                value {
                  number_value: 3.710728303532493
                }
              }
            }
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Feature Scores and P-Values"
        properties {
          key: "p_values"
          value: STRUCT
        }
        properties {
          key: "scores"
          value: STRUCT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
feature_selector.outputs["feature_selection"].get()[0].scores

0.0